In [2]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import os

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler

In [3]:
#Open csv file
data = pd.read_csv(r"/home/kchen/Documents/leak/Other/Data/pan/pan.csv", index_col="CASEID", na_values = -99, low_memory=False)
data.head()

,Unnamed: 0,PAN_LAPTHOR,PAN_JAUNDICE,PAN_JAUNDICE_UNK,PAN_BILIARYSTENT,PAN_CHEMO,PAN_CHEMO_UNK,PAN_RADIO,PAN_RADIO_UNK,PAN_INTRA_ANTIBIOTICS,...,NNEURODEF,NEURODEF,DNEURODEF,NOTHGRAFL,OTHGRAFL,DOTHGRAFL,DSDTOHD,READMISSION,UNPLANREADMISSION,REOPERATION
CASEID,,,,,,,,,,,,,,,,,,,,,
6519196,0,NaN,No,NaN,No stent at time of surgery,No,NaN,No,NaN,2nd or 3rd generation cephalosporin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6522035,1,NaN,Yes,NaN,No stent at time of surgery,No,NaN,No,NaN,2nd or 3rd generation cephalosporin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6523494,2,NaN,Yes,NaN,Endoscopic stent,Yes,NaN,No,NaN,1st generation cephalosporin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6537316,3,NaN,No,NaN,No stent at time of surgery,No,NaN,No,NaN,1st generation cephalosporin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6537525,4,NaN,No,NaN,No stent at time of surgery,No,NaN,No,NaN,2nd or 3rd generation cephalosporin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.shape

(39779, 371)

In [5]:
features = ['PRNCPTX', 'CPT', 'CONCPT1', 'OTHERCPT1', 'OPERYR', 'SEX', 'AGE', 'HEIGHT', 'WEIGHT', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'TRANST','WNDCLAS','ASACLAS', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'PRSEPIS', 'PRSODM', 'PRBUN', 'PRCREAT',  'PRALBUM', 'PRBILI', 'PRALKPH', 'PRWBC', 'PRHCT', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT', 'SEPSHOCKPATOS', 'SSSIPATOS', 'DSSIPATOS', 'OSSIPATOS', 'PNAPATOS', 'VENTPATOS', 'UTIPATOS', 'SEPSISPATOS', 'SEPSHOCKPATOS', 'OPTIME', 'OUPNEUMO', 'CDARREST', 'CDMI', 'SUPINFEC', 'WNDINFD', 'ORGSPCSSI', 'URNINFEC', 'OTHDVT', 'PULEMBOL', 'RENAINSF', 'OPRENAFL', 'DEHIS', 'REINTUB', 'FAILWEAN', 'CNSCVA', 'OTHSYSEP', 'OTHBLEED', 'OTHSESHOCK', 'DOPERTOD','OTHBLEED']

In [6]:
pan_features = ['PAN_JAUNDICE', 'PAN_BILIARYSTENT', 'PAN_CHEMO', 'PAN_RADIO', 'PAN_INTRA_ANTIBIOTICS', 'PAN_APPROACH', 'PAN_OINCIS_TYPE', 'PAN_WOUNDPROT', 'PAN_DUCTSIZE', 'PAN_GLANDTEXT', 'PAN_RECONSTRUCTION', 'PAN_GASTDUO', 'PAN_DRAINS', 'PAN_DRAINS_TYPE', 'PAN_DRAINSYS_TYPE', 'PAN_DRAINSYS_SUCTN', 'PAN_RESECTION', 'PAN_AMYLASE_POD1', 'PAN_AMYLASE_POD230', 'DAMYLASE', 'DDRAINREMOVAL', 'PAN_DRAIN_POD30', 'PAN_FISTULA', 'PAN_DELGASTRIC_20140315', 'PAN_PERCDRAIN_20140315', 'PAN_PERCDRAINAGE', 'PAN_NPWT', 'PAN_POAP', 'PAN_MALIG_HISTOLOGIC', 'PAN_TSTAGE', 'PAN_NSTAGE', 'PAN_MSTAGE', 'PAN_BENIGN_HISTOLOGIC', 'PAN_BENIGN_TUMORSIZE']

In [7]:
#Remove unwanted columns
data = data[features + pan_features]

In [8]:
whipple = data[(data['CPT'] == 48150) | (data['CPT'] == 48153) ]

In [9]:
whipple['PAN_FISTULA'].value_counts(normalize=True)

No                                                          0.546700
No evidence of Biochemical Leak or POPF                     0.260571
Yes-persistent drainage, drain continued >7 days            0.052694
Yes, Grade B POPF present                                   0.037761
Yes-clinical diagnosis, percutaneous drainage performed     0.020779
Biochemical Leak only                                       0.020779
Yes-clinical diagnosis, drain continued >7 days             0.016811
Yes-persistent drainage, percutaneous drainage performed    0.013824
Unknown                                                     0.008704
Yes-clinical diagnosis, reoperation performed               0.005461
Yes, Grade C POPF present                                   0.004608
Yes-clinical diagnosis, spontaneous wound drainage          0.003456
Yes-clinical diagnosis, NPO-TPN                             0.003072
Yes-persistent drainage, NPO-TPN                            0.002901
Yes-persistent drainage, reoperati

In [10]:
data['PAN_FISTULA'].value_counts(normalize=True)

No                                                          0.541567
No evidence of Biochemical Leak or POPF                     0.262827
Yes-persistent drainage, drain continued >7 days            0.056613
Yes, Grade B POPF present                                   0.037105
Biochemical Leak only                                       0.023781
Yes-clinical diagnosis, percutaneous drainage performed     0.019458
Yes-clinical diagnosis, drain continued >7 days             0.018301
Yes-persistent drainage, percutaneous drainage performed    0.013776
Unknown                                                     0.009603
Yes-clinical diagnosis, reoperation performed               0.004073
Yes, Grade C POPF present                                   0.003972
Yes-clinical diagnosis, spontaneous wound drainage          0.002816
Yes-persistent drainage, NPO-TPN                            0.002388
Yes-clinical diagnosis, NPO-TPN                             0.002237
Yes-persistent drainage, reoperati

In [11]:
whipple.shape

(23437, 106)

In [12]:
data = whipple

In [13]:
#Create BMI column
lbs_to_kg_ratio = 0.453592
inch_to_meter_ratio = 0.0254

data['HEIGHT'] *= inch_to_meter_ratio
data['WEIGHT'] *= lbs_to_kg_ratio

data['BMI'] = data['WEIGHT']/(data['HEIGHT']**2)
print(min(data['BMI']))
print(max(data['BMI']))

8.895975179705461
69.9101134832208


In [14]:
print(data.columns.tolist())

['PRNCPTX', 'CPT', 'CONCPT1', 'OTHERCPT1', 'OPERYR', 'SEX', 'AGE', 'HEIGHT', 'WEIGHT', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'TRANST', 'WNDCLAS', 'ASACLAS', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'PRSEPIS', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRBILI', 'PRALKPH', 'PRWBC', 'PRHCT', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT', 'SEPSHOCKPATOS', 'SSSIPATOS', 'DSSIPATOS', 'OSSIPATOS', 'PNAPATOS', 'VENTPATOS', 'UTIPATOS', 'SEPSISPATOS', 'SEPSHOCKPATOS', 'OPTIME', 'OUPNEUMO', 'CDARREST', 'CDMI', 'SUPINFEC', 'WNDINFD', 'ORGSPCSSI', 'URNINFEC', 'OTHDVT', 'PULEMBOL', 'RENAINSF', 'OPRENAFL', 'DEHIS', 'REINTUB', 'FAILWEAN', 'CNSCVA', 'OTHSYSEP', 'OTHBLEED', 'OTHSESHOCK', 'DOPERTOD', 'OTHBLEED', 'PAN_JAUNDICE', 'PAN_BILIARYSTENT', 'PAN_CHEMO', 'PAN_RADIO', 'PAN_INTRA_ANTIBIOTICS', 'PAN_APPROACH', 'PAN_OINCIS_TYPE', 'PAN_WOUNDPROT', 'PAN_DUCTSIZE', 'PAN_GLANDTEXT', 'PAN_RECONSTRU

In [15]:
data.shape

(23437, 107)

In [16]:
data['PAN_FISTULA'].value_counts()

No                                                          12813
No evidence of Biochemical Leak or POPF                      6107
Yes-persistent drainage, drain continued >7 days             1235
Yes, Grade B POPF present                                     885
Yes-clinical diagnosis, percutaneous drainage performed       487
Biochemical Leak only                                         487
Yes-clinical diagnosis, drain continued >7 days               394
Yes-persistent drainage, percutaneous drainage performed      324
Unknown                                                       204
Yes-clinical diagnosis, reoperation performed                 128
Yes, Grade C POPF present                                     108
Yes-clinical diagnosis, spontaneous wound drainage             81
Yes-clinical diagnosis, NPO-TPN                                72
Yes-persistent drainage, NPO-TPN                               68
Yes-persistent drainage, reoperation performed                 44
Name: PAN_

In [17]:
data.loc[data['PAN_FISTULA'] == 'No evidence of Biochemical Leak or POPF', 'PAN_FISTULA'] = False
data.loc[data['PAN_FISTULA'] == 'No', 'PAN_FISTULA'] = False
data.loc[data['PAN_FISTULA'] == 'Biochemical Leak only', 'PAN_FISTULA'] = False

data.loc[data['PAN_FISTULA'] == 'Yes-persistent drainage, drain continued >7 days', 'PAN_FISTULA'] = True
data.loc[data['PAN_FISTULA'] == 'Yes, Grade B POPF present', 'PAN_FISTULA'] = True
data.loc[data['PAN_FISTULA'] == 'Yes-clinical diagnosis, percutaneous drainage performed', 'PAN_FISTULA'] = True
data.loc[data['PAN_FISTULA'] == 'Yes-persistent drainage, percutaneous drainage performed', 'PAN_FISTULA'] = True
data.loc[data['PAN_FISTULA'] == 'Yes-clinical diagnosis, reoperation performed', 'PAN_FISTULA'] = True
data.loc[data['PAN_FISTULA'] == 'Yes, Grade C POPF present', 'PAN_FISTULA'] = True
data.loc[data['PAN_FISTULA'] == 'Yes-clinical diagnosis, spontaneous wound drainage', 'PAN_FISTULA'] = True
data.loc[data['PAN_FISTULA'] == 'Yes-persistent drainage, NPO-TPN', 'PAN_FISTULA'] = True
data.loc[data['PAN_FISTULA'] == 'Yes-clinical diagnosis, NPO-TPN', 'PAN_FISTULA'] = True
data.loc[data['PAN_FISTULA'] == 'Yes-persistent drainage, reoperation performed', 'PAN_FISTULA'] = True
data.loc[data['PAN_FISTULA'] == 'Yes-clinical diagnosis, drain continued >7 days', 'PAN_FISTULA'] = True

data['PAN_FISTULA'].value_counts()

False      19407
True        3826
Unknown      204
Name: PAN_FISTULA, dtype: int64

In [18]:
#Remove unwanted indication from COL_INDICATION
data.drop(data[data['PAN_FISTULA'] == 'Unknown'].index, inplace=True)
data['PAN_FISTULA'].value_counts()

False    19407
True      3826
Name: PAN_FISTULA, dtype: int64

In [19]:
data['PAN_PERCDRAINAGE'] = data['PAN_PERCDRAINAGE'].fillna(False)
data['PAN_PERCDRAINAGE'].value_counts(dropna=False)

False                                                20088
Yes-other                                              901
Yes-pus                                                832
Yes-amylase-rich fluid                                 666
Yes-bile                                               273
Yes-amylase-rich fluid,Yes-pus                         263
Yes-pus,Yes-bile                                        44
Yes-pus,Yes-other                                       40
Yes-amylase-rich fluid,Yes-other                        35
Yes-amylase-rich fluid,Yes-bile                         27
Yes-amylase-rich fluid,Yes-pus,Yes-bile                 27
Yes-bile,Yes-other                                      17
Yes-pus,Yes-bile,Yes-other                               9
Yes-amylase-rich fluid,Yes-pus,Yes-other                 8
Yes-amylase-rich fluid,Yes-pus,Yes-bile,Yes-other        2
Yes-amylase-rich fluid,Yes-bile,Yes-other                1
Name: PAN_PERCDRAINAGE, dtype: int64

In [20]:
data.loc[data['PAN_PERCDRAINAGE'] != False, 'PAN_PERCDRAINAGE'] = True
data['PAN_PERCDRAINAGE'].value_counts(dropna=False)

False    20088
True      3145
Name: PAN_PERCDRAINAGE, dtype: int64

In [21]:
data['PAN_POAP'].fillna(False, inplace=True)
data.loc[data['PAN_POAP'] == 'No', 'PAN_POAP'] = False
data.loc[data['PAN_POAP'] == 'Unknown', 'PAN_POAP'] = False
data.loc[data['PAN_POAP'] != False, 'PAN_POAP'] = True
data['PAN_POAP'].value_counts(dropna=False)

/home/kchen/.local/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


True     23044
False      189
Name: PAN_POAP, dtype: int64

In [22]:
data['PAN_POAP'] = data['PAN_POAP'].astype('bool')

In [23]:
data['PAN_POAP'].value_counts(dropna=False)

True     23044
False      189
Name: PAN_POAP, dtype: int64

In [24]:
#Cast ASA class as ordered categorical
cat_type = CategoricalDtype(categories=['None assigned','1-No Disturb','2-Mild Disturb','3-Severe Disturb','4-Life Threat','5-Moribund'], ordered=True)
data['ASACLAS'].astype(cat_type)


CASEID
6522035    3-Severe Disturb
6523494    3-Severe Disturb
6537888    3-Severe Disturb
6540993    3-Severe Disturb
6545348    3-Severe Disturb
                 ...       
6432651      2-Mild Disturb
6433086    3-Severe Disturb
6433351    3-Severe Disturb
6433506       4-Life Threat
6433572    3-Severe Disturb
Name: ASACLAS, Length: 23233, dtype: category
Categories (6, object): ['None assigned' < '1-No Disturb' < '2-Mild Disturb' < '3-Severe Disturb' < '4-Life Threat' < '5-Moribund']

In [25]:
#Convert 90+ to 91 and AGE to int32
data.loc[data['AGE'] == '90+', 'AGE'] = 91
pd.to_numeric(data['AGE'], downcast='integer')

CASEID
6522035    56
6523494    76
6537888    54
6540993    57
6545348    70
           ..
6432651    62
6433086    73
6433351    69
6433506    75
6433572    64
Name: AGE, Length: 23233, dtype: int8

In [26]:
data['PAN_APPROACH'].value_counts()

Open (planned)                                        21172
Robotic                                                 849
Laparoscopic                                            364
Laparoscopic w/ unplanned conversion to open            286
Laparoscopic w/ open assist                             223
Robotic w/ unplanned conversion to open                 148
Robotic w/ open assist                                  106
Hybrid                                                   55
Other MIS approach                                       13
Hybrid w/ open assist                                    10
Other                                                     2
Hybrid w/ unplanned conversion to open                    2
Unknown                                                   1
NOTES w/ unplanned conversion to open                     1
Other MIS approach w/ unplanned conversion to open        1
Name: PAN_APPROACH, dtype: int64

In [27]:
data.loc[data['PAN_APPROACH'] == 'Laparoscopic', 'PAN_APPROACH'] = 'Laparoscopic'
data.loc[data['PAN_APPROACH'] == 'Laparoscopic w/ open assist', 'PAN_APPROACH'] = 'Laparoscopic'
data.loc[data['PAN_APPROACH'] == 'Laparoscopic w/ unplanned conversion to open', 'PAN_APPROACH'] = 'Laparoscopic'

data.loc[data['PAN_APPROACH'] == 'Robotic', 'PAN_APPROACH'] = 'Laparoscopic'
data.loc[data['PAN_APPROACH'] == 'Robotic w/ open assist', 'PAN_APPROACH'] = 'Laparoscopic'
data.loc[data['PAN_APPROACH'] == 'Robotic w/ unplanned conversion to open', 'PAN_APPROACH'] = 'Laparoscopic'

data.loc[data['PAN_APPROACH'] == 'NOTES', 'PAN_APPROACH'] = 'Laparoscopic'
data.loc[data['PAN_APPROACH'] == 'NOTES w/ open assist', 'PAN_APPROACH'] = 'Laparoscopic w/ open assist'
data.loc[data['PAN_APPROACH'] == 'NOTES w/ unplanned conversion to open', 'PAN_APPROACH'] = 'Laparoscopic'

data.loc[data['PAN_APPROACH'] == 'Other MIS approach', 'PAN_APPROACH'] = 'Laparoscopic'
data.loc[data['PAN_APPROACH'] == 'Other MIS approach w/ open assist', 'PAN_APPROACH'] = 'Laparoscopic'
data.loc[data['PAN_APPROACH'] == 'Other MIS approach w/ unplanned conversion to open', 'PAN_APPROACH'] = 'Laparoscopic'

data.loc[data['PAN_APPROACH'] == 'Hybrid', 'PAN_APPROACH'] = 'Laparoscopic'
data.loc[data['PAN_APPROACH'] == 'Hybrid w/ open assist', 'PAN_APPROACH'] = 'Laparoscopic'
data.loc[data['PAN_APPROACH'] == 'Hybrid w/ unplanned conversion to open', 'PAN_APPROACH'] = 'Laparoscopic'

data.loc[data['PAN_APPROACH'] == 'SILS', 'PAN_APPROACH'] = 'Laparoscopic'

data.loc[data['PAN_APPROACH'] == 'Other', 'PAN_APPROACH'] = 'Unknown'

data['PAN_APPROACH'].value_counts()

Open (planned)    21172
Laparoscopic       2058
Unknown               3
Name: PAN_APPROACH, dtype: int64

In [28]:
data.to_csv(r'/home/kchen/Documents/leak/Other/Data/pan/clean_pan.csv')

In [4]:
data = pd.read_csv(r'/home/kchen/Documents/leak/old_leak/Other/Data/pan/clean_pan.csv')

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3427: DtypeWarning: Columns (4,99) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
drop = ['PRPT','OUPNEUMO', 'CDARREST', 'CDMI', 'SUPINFEC', 'WNDINFD', 'ORGSPCSSI', 'URNINFEC', 'OTHDVT', 'PULEMBOL', 'RENAINSF', 'OPRENAFL', 'DEHIS', 'REINTUB', 'FAILWEAN', 'CNSCVA', 'OTHSYSEP', 'OTHBLEED', 'OTHSESHOCK', 'DOPERTOD','PAN_AMYLASE_POD230']
data.drop(drop, axis=1, inplace=True)

In [6]:
num_candidates = []
for col in data.columns:
    if len(data[col].unique()) > 100:
        num_candidates.append(col)

In [7]:
num_cols = ['AGE','HEIGHT','WEIGHT', 'BMI','OPTIME','PRSODM','PRBUN','PRCREAT','PRALBUM','PRBILI','PRALKPH','PRWBC','PRHCT','PRPLATE','PRPTT','PRINR','OPTIME','PAN_AMYLASE_POD1']

In [8]:
cat_cols = list(set(data.columns) - set(num_cols) - set('PAN_FISTULA'))
for col in cat_cols:
    data[col].fillna(value='Unknown', inplace=True)


In [9]:
num_imputer = SimpleImputer(strategy = 'median')
# Imputation for numeric:
for ncol in num_cols:
    data[ncol] = num_imputer.fit_transform(data[ncol].values.reshape(-1, 1))


In [10]:
for col in num_cols:
    data[col] = RobustScaler().fit_transform(data[col].values.reshape(-1,1))

In [11]:
le=LabelEncoder()
for col in cat_cols:
        data[col] = le.fit_transform(data[col].astype(str))
data.head()


,CASEID,PRNCPTX,CPT,CONCPT1,OTHERCPT1,OPERYR,SEX,AGE,HEIGHT,WEIGHT,...,PAN_PERCDRAINAGE,PAN_NPWT,PAN_POAP,PAN_MALIG_HISTOLOGIC,PAN_TSTAGE,PAN_NSTAGE,PAN_MSTAGE,PAN_BENIGN_HISTOLOGIC,PAN_BENIGN_TUMORSIZE,BMI
0,11345,0,0,190,459,3,1,-0.666667,1.333333,1.240741,...,0,2,1,9,4,0,0,7,3,0.497005
1,11348,0,0,190,406,3,1,0.666667,0.666667,-0.055556,...,0,2,1,3,3,1,0,7,3,-0.427988
2,11390,0,0,190,272,3,0,-0.800000,-0.333333,0.574074,...,0,2,1,9,3,0,0,7,3,1.057034
3,11401,1,1,190,459,3,1,-0.600000,0.500000,0.574074,...,0,2,1,1,2,0,0,7,3,0.382758
4,11406,0,0,190,191,3,0,0.266667,-0.500000,-0.351852,...,0,2,1,9,3,0,2,7,3,-0.046548


In [12]:
outcomes = ['PAN_PERCDRAINAGE','PAN_DRAIN_POD30','DDRAINREMOVAL','PAN_PERCDRAIN_20140315','PAN_DELGASTRIC_20140315','DAMYLASE','PAN_POAP']
data.drop(outcomes, axis=1, inplace=True)

In [13]:
data.to_csv('clean_pan.csv')